In [1]:
# imports
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import seaborn as sns
from sklearn import (datasets, neighbors,
                     model_selection as skms,
                     linear_model, metrics)

In [9]:
tp53_somatic_muts = pd.read_csv("data/geneVariationsAllIARC_TP53_Database__R20.txt", sep='\t')
display(tp53_somatic_muts)

,MUT_ID,hg18_Chr17_coordinates,hg19_Chr17_coordinates,hg38_Chr17_coordinates,ExonIntron,Codon_number,Description,c_description,g_description,g_description_GRCh38,...,h1433snWT_Saos2,AIP1nWT_Saos2,PUMAnWT_Saos2,SubG1nWT_Saos2,Type_ID,AAchange_ID,MutationView_ID,Somatic_count,Germline_count,CellLine_count
0,2,7520634,7579909,7676591,2-exon,2,G>A,c.4G>A,g.7579909C>T,g.7676591C>T,...,NaN,NaN,NaN,NaN,9,561,19345,0,0,0
1,3,7520634,7579909,7676591,2-exon,2,G>C,c.4G>C,g.7579909C>G,g.7676591C>G,...,NaN,NaN,NaN,NaN,11,562,19346,0,0,0
2,5,7520633,7579908,7676590,2-exon,2,A>C,c.5A>C,g.7579908T>G,g.7676590T>G,...,NaN,NaN,NaN,NaN,6,557,19339,0,0,0
3,6,7520633,7579908,7676590,2-exon,2,A>G,c.5A>G,g.7579908T>C,g.7676590T>C,...,NaN,NaN,NaN,NaN,7,560,19343,0,0,0
4,7,7520633,7579908,7676590,2-exon,2,A>T,c.5A>T,g.7579908T>A,g.7676590T>A,...,NaN,NaN,NaN,NaN,8,563,19347,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7988,29623,7520075,7579350,7676032,4-exon,113,del3ins3,c.337_339delinsCCT,g.7579348_7579350delinsAGG,g.7676030_7676032delinsAGG,...,NaN,NaN,NaN,NaN,5,6056,10023,0,0,0
7989,29628,7520056,7579331,7676013,4-exon,119,CC>GT,c.356_357CC>GT,g.7579330_7579331delinsAC,g.7676012_7676013delinsAC,...,NaN,NaN,NaN,NaN,3,3,18367,0,0,0
7990,29817,7520199,7579474,7676156,4-exon,71,CC>TT,c.213_214CC>TT,g.7579473_7579474delinsAA,g.7676155_7676156delinsAA,...,NaN,NaN,NaN,NaN,4,3304,9717,0,0,0
7991,29826,7518254,7577529,7674211,7-exon,251,TC>AT,c.752_753TC>AT,g.7577528_7577529delinsAT,g.7674210_7674211delinsAT,...,NaN,NaN,NaN,NaN,3,1038,20190,0,0,0


In [10]:
prevalence_data = pd.read_csv("data/PrevalenceDownload_r20.csv")
display(prevalence_data)

,Prevalence_ID,Topography,Topo_code,Morphology,Morpho_code,Sample_analyzed,Sample_mutated,Population,Country,Comment,...,exon4,exon5,exon6,exon7,exon8,exon9,exon10,exon11,Exclude_analysis,WGS_WXS
0,1,ESOPHAGUS,C15,"Squamous cell carcinoma, NOS",8070/3,14,5,Western Europe,France,NaN,...,False,True,True,True,True,True,False,False,False,No
1,2,ESOPHAGUS,C15,"Cancer, NOS",8000/3,10,5,Eastern Asia,China,NaN,...,False,True,True,True,True,False,False,False,False,No
2,3,ESOPHAGUS,C15,"Squamous cell carcinoma, NOS",8070/3,34,15,South America,Uruguay,NaN,...,False,True,True,True,True,False,False,False,False,No
3,4,STOMACH,C16,"Cancer, NOS",8000/3,14,9,Eastern Asia,Japan,NaN,...,True,True,True,True,True,False,False,False,False,No
4,5,PANCREAS,C25,"Adenocarcinoma, NOS",8140/3,14,6,Eastern Asia,Japan,NaN,...,False,True,True,True,True,False,False,False,False,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2382,2553,OVARY,C56,"Serous cystadenocarcinoma, NOS (C56.9)",8441/3,126,119,Australia/New Zealand,Australia,High grade cases,...,True,True,True,True,True,True,True,True,False,No
2383,2554,OVARY,C56,"Serous cystadenocarcinoma, NOS (C56.9)",8441/3,4,2,Australia/New Zealand,Australia,Low grade cases,...,True,True,True,True,True,True,True,True,False,No
2384,2555,OVARY,C56,"Serous cystadenocarcinoma, NOS (C56.9)",8441/3,15,13,Australia/New Zealand,Australia,Early stage cases,...,True,True,True,True,True,True,True,True,False,No
2385,2556,ADRENAL GLAND,C74,"Pheochromocytoma, malignant (C74.1)",8700/3,6,5,South Eastern Asia,Chinese Taipei (Taiwan),NaN,...,True,True,True,True,True,True,True,True,True,No


In [14]:
(prevalence_train_features, prevalence_test_features,
 prevalence_train_target, prevalence_test_target) = skms.train_test_split(prevalence_data, prevalence_data["Morphology"], test_size = 0.25)

print("train features shape: ", prevalence_train_features.shape)
print("train features:\n", prevalence_train_features, "\n")
print("test features shape: ", prevalence_test_features.shape)
print("test features:\n", prevalence_test_features, "\n")
print("train target shape: ", prevalence_train_target.shape)
print("train target:\n", prevalence_train_target, "\n")
print("test target shape: ", prevalence_test_target.shape)
print("test target:\n", prevalence_test_target, "\n")

train features shape:  (1790, 37)
train features:
       Prevalence_ID                                     Topography Topo_code  \
605             654                                        BLADDER       C67   
1725           1872                                COLORECTUM, NOS   C18-C20   
84               91                                      ESOPHAGUS       C15   
963            1027                                          BRAIN       C71   
2123           2287              LIVER AND INTRAHEPATIC BILE DUCTS       C22   
...             ...                                            ...       ...   
1775           1927          OTHER AND UNSPECIFIED PARTS OF TONGUE       C02   
1406           1527  HEMATOPOIETIC AND RETICULOENDOTHELIAL SYSTEMS       C42   
2225           2391                                  ADRENAL GLAND       C74   
2088           2251                                          BRAIN       C71   
754             813                                          BRAIN   